In [1]:
import os 
import sys
import pickle

In [2]:
%pwd

'e:\\ML_Projects\\Predictive_Maintenance\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\ML_Projects\\Predictive_Maintenance'

In [5]:
from src.exception import CustomException
from src.logger import logging
import pandas as pd
import numpy as np

In [6]:
from dataclasses import dataclass
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [7]:
@dataclass
class DataTransformationConfig:
    preprocessor_obj_file_path = os.path.join("artifacts", "preprocessor.pkl")

In [8]:
def save_object(file_path, obj):
    
    try:
        dir_path = os.path.dirname(file_path)
        
        os.makedirs(dir_path, exist_ok = True)
        
        with open(file_path, "wb") as file_obj:
            pickle.dump(obj, file_obj)
                   
    except Exception as e:
        raise CustomException(e, sys)

In [9]:
class DataTransformation:
    def __init__(self):
        self.data_transformation_config = DataTransformationConfig()
        
    def get_data_trasnformer_object(self):
        try:
            num_cols = ['Engine_no', 'Cycle_no',
            'LPC_outlet_temperature',
            'HPC_outlet_temperature', 'LPT_outlet_temperature',
            'HPC_outlet_pressure',
            'Physical_fan_speed', 'Physical_core_speed', 
            'HPC_outlet_static_pressure', 'Fuel_flow_ratio', 'Fan_speed',
            'Bypass_ratio', 'Bleed_enthalpy',
            'High_pressure_cool_air_flow', 'Low_pressure_cool_air_flow']
            
            num_pipeline = Pipeline(
                
                steps = [
                    ("scaler", RobustScaler()),
                    ("PCA", PCA(n_components=8))
                    
                ]
            )
            
            logging.info("Pipeline created")
            
            preprocessor = ColumnTransformer(
                
                [
                    ("num_pipeline", num_pipeline, num_cols)
                ]
            )
            
            return preprocessor
        
        except Exception as e:
            raise CustomException(e, sys)
        
    def initiate_data_transformation(self, train_path, test_path):
        
        try:
            train_df = pd.read_csv(train_path)
            test_df = pd.read_csv(test_path)
            
            train_df["RUL"][train_df["RUL"] > 103] = 103
            test_df["RUL"][test_df["RUL"] > 103] = 103
            
            logging.info("Train & test data readed")
            
            target_column_name = "RUL"
            drop_cols = [
                'Setting_1', 'Setting_2', 'Setting_3', 'Fan_inlet_temperature', 'Fan_inlet_pressure', 
                'Bypass_duct_pressure', 'Engine_pressure_ratio', 'Core_speed', 'Burner_fuel_air_ratio', 
                'Required_fan_speed', 'Required_fan_conversion_speed']  
            
            
            input_feature_train_df = train_df.drop(columns=[target_column_name] + drop_cols)
            target_feature_train_df = train_df[target_column_name]
            
            input_feature_test_df = test_df.drop(columns=[target_column_name] + drop_cols)
            target_feature_test_df = test_df[target_column_name]
            
            
            preprocessor_obj = self.get_data_trasnformer_object()
            logging.info("Applying preprocessing obj on train & test dataframe")
            
        #Transforming into Preprocessor object to data
            input_feature_train_arr = preprocessor_obj.fit_transform(input_feature_train_df) #train data
            input_feature_test_arr = preprocessor_obj.transform(input_feature_test_df) #test data
            logging.info("Preprocessing applied to training & test datasets") 
            
            #Converting into numpy array for train & test data
            train_arr = np.c_[input_feature_train_arr, np.array(target_feature_train_df)] #train array 
            test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)] #test array
            logging.info("Data transfom into the array")
            
            #function from utils to save the preprocessor file
            save_object(
                file_path=self.data_transformation_config.preprocessor_obj_file_path,
                obj = preprocessor_obj
            )
            logging.info("Preprocessor file saved as Pickle file")
            
            return (
                train_arr,
                test_arr
            )
            
            
        except Exception as e:
            raise CustomException(e, sys)

In [10]:
from src.components.data_ingestion import DataIngestion

In [11]:
if __name__ == "__main__":
    obj = DataIngestion()
    train_data, test_data = obj.initiate_data_ingestion()
    
    data_transformation = DataTransformation()
    data_transformation.initiate_data_transformation(train_data, test_data)

e:\ML_Projects\Predictive_Maintenance\src\utils.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_train = pd.read_sql_query("select * from train1", mydb)
e:\ML_Projects\Predictive_Maintenance\src\utils.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql_query("select * from test1", mydb)
C:\Users\Ayush Gandhi\AppData\Local\Temp\ipykernel_13324\3415109918.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["RUL"][train_df["RUL"] > 103] = 103
C:\Users\Ayush 